In [ ]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import glob

import numpy as np
import sklearn.metrics as skm

# Generate command to generate predictions

In [ ]:
data_dir = '.'
conf_f = os.path.join(data_dir, 'f_channel_1.csv')  # old: f_channel.csv
conf_c = os.path.join(data_dir, 'c_channel_1.csv')  # old: c_channel.csv

In [ ]:
conf_f_df = pd.read_csv(conf_f)
conf_f_df = conf_f_df.iloc[:,1:]
conf_f_df

In [ ]:
conf_c_df = pd.read_csv(conf_c)
conf_c_df = conf_c_df.iloc[:,1:]
conf_c_df

In [ ]:
local_data_dir = './processed'
map_db_dir = {
    'mass_ss1': 'mass_ss1',
    'mass_ss2': 'mass_ss2',
    'mass_ss3': 'mass_ss3',
    'mass_ss4': 'mass_ss4',
    'mass_ss5': 'mass_ss5',
    'SleepEDF': 'sedf_sc',
    'isruc1': 'isruc_sg1', 
    'shhs1_normal': 'shhs1_nor',
    'shhs1_osa': 'shhs1_osa',
}

In [ ]:
base_cmd = 'cmd'
commands = []
count = 0
for conf_df in [conf_f_df, conf_c_df]:
    base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
    for _, b_row in base_df.iterrows():
        # Base dataset
        b_db = str(b_row['source_dataset']).lower()
        b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
        b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

        # Base dataset
        each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
        tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})
        
        # Initialize command
        src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"
        
        for _, t_row in tran_df.iterrows():
            t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
            count += len(t_chs)
            t_chs = '|'.join(t_chs)
            t_chs = t_chs.replace('EEG ','').replace(' ', '-')
            t_db = str(t_row['transfer_dataset']).lower()
            t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
            tgt_proj = f"{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"
            
            check1 = osp.isdir(f"tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")
            check2 = osp.isfile(f"{src_proj}/model/model_weights.h5")
            check3 = osp.isfile(f"{tgt_proj}/model/model_weights.h5")
            if check1 and check2 and check3:
                print(f"cd tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type3 --weights_file_name '../{src_proj}/model/model_weights.h5' --overwrite")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type6 --weights_file_name '../{tgt_proj}/model/model_weights.h5' --overwrite")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type9 --overwrite")
                print("cd ..")
            # else:
            #     print(f" ** Skip tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")

# Calculate performance metrics

In [ ]:
results = []
for conf_df in [conf_f_df, conf_c_df]:
    base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
    for _, b_row in base_df.iterrows():
        # Base dataset
        b_db = str(b_row['source_dataset']).lower()
        b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
        b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

        # Base dataset
        each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
        tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})
        
        # Initialize command
        src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"
        
        for _, t_row in tran_df.iterrows():
            t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
            count += len(t_chs)
            t_chs = '|'.join(t_chs)
            t_chs = t_chs.replace('EEG ','').replace(' ', '-')
            t_db = str(t_row['transfer_dataset']).lower()
            t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
            tgt_proj = f"{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"
            
            a_result = [
                f"{b_db.lower().replace(' ','_')}", f"eeg_{b_ch.lower().replace('-','_')}",
                f"{t_db.lower().replace(' ','_')}", f"eeg_{t_chs.lower().replace('-','_')}",
            ]
            for t in [3,6,9]:
                scan_gt_path = osp.join(
                    f"predictions/{src_proj}__to__{tgt_proj}_type{t}", 'test_data', 'dataset_1', '*_TRUE.npy'
                )
                gt_files = list(glob.glob(scan_gt_path))
                if len(gt_files) > 0:
                    y_trues = []
                    y_preds = []
                    for fp_gt in gt_files:
                        fp_pred = fp_gt.replace('_TRUE.npy', '_PRED.npy')
                        y_true = np.squeeze(np.load(fp_gt))
                        y_pred = np.load(fp_pred)
                        y_trues.extend(y_true)
                        y_preds.extend(y_pred)
                    y_trues, y_preds = np.array(y_trues), np.array(y_preds)
                    a_result.extend([
                        skm.accuracy_score(y_trues, y_preds),
                        skm.f1_score(y_trues, y_preds, average='macro')
                    ])
                    # print(f"{src_proj}__to__{tgt_proj}_type{t}")
                    # print(y_trues.shape, y_preds.shape)
                    # print(f"ACC: {skm.accuracy_score(y_trues, y_preds)}")
                    # print(f"MF1: {skm.f1_score(y_trues, y_preds, average='macro')}")
                    # print(f"PR: {skm.precision_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                    # print(f"RE: {skm.recall_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                    # print(f"F1: {skm.f1_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
            results.append(a_result)
            
tf_df = pd.DataFrame(results, columns=[
    'source_dataset','source_channel','target_dataset','target_channel',
    'acc_type3','mf1_type3','acc_type6','mf1_type6','acc_type9','mf1_type9'])

In [ ]:
tf_df

In [ ]:
# tf_df.to_csv('utime_conclusions.csv', index=False)

In [ ]:
# Run more with other weights for statistical tests

results = []
for temp in [2,3]:
    for conf_df in [conf_f_df, conf_c_df]:
        base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
        for _, b_row in base_df.iterrows():
            # Base dataset
            b_db = str(b_row['source_dataset']).lower()
            b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
            b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

            # Base dataset
            each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
            tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})

            # Initialize command
            src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"

            for _, t_row in tran_df.iterrows():
                t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
                t_chs = '|'.join(t_chs)
                t_chs = t_chs.replace('EEG ','').replace(' ', '-')
                t_db = str(t_row['transfer_dataset']).lower()
                t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
                tgt_proj = f"{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"

                a_result = [
                    f"{b_db.lower().replace(' ','_')}", f"eeg_{b_ch.lower().replace('-','_')}",
                    f"{t_db.lower().replace(' ','_')}", f"eeg_{t_chs.lower().replace('-','_')}",
                ]

                for t in [3,6,9]:
                    scan_gt_path = osp.join(
                        f"predictions/{src_proj}__to__{tgt_proj}_type{t}_{temp}", 'test_data', 'dataset_1', '*_TRUE.npy'
                    )
                    gt_files = list(glob.glob(scan_gt_path))
                    if len(gt_files) > 0:
                        y_trues = []
                        y_preds = []
                        for fp_gt in gt_files:
                            fp_pred = fp_gt.replace('_TRUE.npy', '_PRED.npy')
                            y_true = np.squeeze(np.load(fp_gt))
                            y_pred = np.load(fp_pred)
                            y_trues.extend(y_true)
                            y_preds.extend(y_pred)
                        y_trues, y_preds = np.array(y_trues), np.array(y_preds)
                        a_result.extend([
                            skm.accuracy_score(y_trues, y_preds),
                            skm.f1_score(y_trues, y_preds, average='macro')
                        ])
                        # print(f"{src_proj}__to__{tgt_proj}_type{t}")
                        # print(y_trues.shape, y_preds.shape)
                        # print(f"ACC: {skm.accuracy_score(y_trues, y_preds)}")
                        # print(f"MF1: {skm.f1_score(y_trues, y_preds, average='macro')}")
                        # print(f"PR: {skm.precision_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                        # print(f"RE: {skm.recall_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                        # print(f"F1: {skm.f1_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                if len(a_result) > 4:
                    results.append(a_result)
            
tf_df = pd.DataFrame(results, columns=[
    'source_dataset','source_channel','target_dataset','target_channel',
    'acc_type3','mf1_type3','acc_type6','mf1_type6','acc_type9','mf1_type9'])

In [ ]:
tf_df.dropna()

In [ ]:
tf_df.to_csv('utime_conclusions_more.csv', index=False)